# Ivy Database

We have a SQL-server based databased named as Ivy Database in the FRE department. Yet the access may be closed on holidays and is only limited to NYU network even if it is available. Here I provide a demo for retrieve data from this database.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pymssql
import pandas as pd
import numpy as np

conn = pymssql.connect(server="fresql.poly.edu", user="ivyuser", password="resuyvi")
sql1 = '''
SELECT sp.Date date, 
sp.SecurityID,
s.Ticker ticker, 
sp.ClosePrice price
FROM XFDATA.DBO.SECURITY_PRICE AS SP
JOIN XFDATA.DBO.SECURITY AS S ON sp.SecurityID = S.SecurityID
WHERE SP.Date BETWEEN '2021-03-01' AND '2021-03-01'
order by 
s.Ticker,  
sp.Date
'''
data = pd.read_sql(sql1, conn)
data

,date,SecurityID,ticker,price
0,2021-03-01,102479,,0.000000
1,2021-03-01,209477,,114.989998
2,2021-03-01,109811,,24.540001
3,2021-03-01,101149,A,124.480003
4,2021-03-01,210354,AA,25.990000
...,...,...,...,...
8721,2021-03-01,128292,ZZZZ,0.000000
8722,2021-03-01,134880,ZZZZ,80.853302
8723,2021-03-01,128350,ZZZZ,0.010000
8724,2021-03-01,129857,ZZZZ,604.389221


Below I follow instructions from [Stefan Jansen's github](https://github.com/stefan-jansen/machine-learning-for-trading) to query data from open resources.

# Select tickers

I first get the list for S&P 500 stocks. I use historical data for these stocks to create my machine learning dataset.

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
df = pd.read_html(url, header=0)[0]
df

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
...,...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [3]:
ticker_list = df['Symbol'].tolist() + ['^GSPC']
ticker_str = " ".join(ticker_list)
ticker_str

'MMM AOS ABT ABBV ABMD ACN ATVI ADM ADBE ADP AAP AES AFL A APD AKAM ALK ALB ARE ALGN ALLE LNT ALL GOOGL GOOG MO AMZN AMCR AMD AEE AAL AEP AXP AIG AMT AWK AMP ABC AME AMGN APH ADI ANSS AON APA AAPL AMAT APTV ACGL ANET AJG AIZ T ATO ADSK AZO AVB AVY BKR BALL BAC BBWI BAX BDX WRB BRK.B BBY BIO TECH BIIB BLK BK BA BKNG BWA BXP BSX BMY AVGO BR BRO BF.B CHRW CDNS CZR CPT CPB COF CAH KMX CCL CARR CTLT CAT CBOE CBRE CDW CE CNC CNP CDAY CF CRL SCHW CHTR CVX CMG CB CHD CI CINF CTAS CSCO C CFG CLX CME CMS KO CTSH CL CMCSA CMA CAG COP ED STZ CEG COO CPRT GLW CTVA CSGP COST CTRA CCI CSX CMI CVS DHI DHR DRI DVA DE DAL XRAY DVN DXCM FANG DLR DFS DISH DIS DG DLTR D DPZ DOV DOW DTE DUK DD DXC EMN ETN EBAY ECL EIX EW EA ELV LLY EMR ENPH ETR EOG EPAM EQT EFX EQIX EQR ESS EL ETSY RE EVRG ES EXC EXPE EXPD EXR XOM FFIV FDS FAST FRT FDX FITB FRC FE FIS FISV FLT FMC F FTNT FTV FBHS FOXA FOX BEN FCX GRMN IT GNRC GD GE GIS GM GPC GILD GL GPN GS HAL HIG HAS HCA PEAK HSIC HSY HES HPE HLT HOLX HD HON HRL HST HWM H

# yfinance

I use `yfinance` to collect data from yahoo finance for the listed stocks above.

In [4]:
import pandas as pd
import yfinance as yf

tickers = yf.Tickers(ticker_str)


In [5]:
tickers.tickers

{'MMM': yfinance.Ticker object <MMM>,
 'AOS': yfinance.Ticker object <AOS>,
 'ABT': yfinance.Ticker object <ABT>,
 'ABBV': yfinance.Ticker object <ABBV>,
 'ABMD': yfinance.Ticker object <ABMD>,
 'ACN': yfinance.Ticker object <ACN>,
 'ATVI': yfinance.Ticker object <ATVI>,
 'ADM': yfinance.Ticker object <ADM>,
 'ADBE': yfinance.Ticker object <ADBE>,
 'ADP': yfinance.Ticker object <ADP>,
 'AAP': yfinance.Ticker object <AAP>,
 'AES': yfinance.Ticker object <AES>,
 'AFL': yfinance.Ticker object <AFL>,
 'A': yfinance.Ticker object <A>,
 'APD': yfinance.Ticker object <APD>,
 'AKAM': yfinance.Ticker object <AKAM>,
 'ALK': yfinance.Ticker object <ALK>,
 'ALB': yfinance.Ticker object <ALB>,
 'ARE': yfinance.Ticker object <ARE>,
 'ALGN': yfinance.Ticker object <ALGN>,
 'ALLE': yfinance.Ticker object <ALLE>,
 'LNT': yfinance.Ticker object <LNT>,
 'ALL': yfinance.Ticker object <ALL>,
 'GOOGL': yfinance.Ticker object <GOOGL>,
 'GOOG': yfinance.Ticker object <GOOG>,
 'MO': yfinance.Ticker object <MO>

In [6]:
data = pd.DataFrame([])

for ticker in ticker_list:

    t = tickers.tickers[ticker].history(period="max")
    t["Ticker"] = ticker

    data = pd.concat([data, t], axis=0)

data

Got error from yahoo api for ticker BRK.B, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- BRK.B: No data found for this date range, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted


,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Adj Close
Date,,,,,,,,,
1962-01-02 00:00:00-05:00,0.000000,0.771044,0.748367,0.754036,2.128000e+05,0.0,0.0,MMM,NaN
1962-01-03 00:00:00-05:00,0.000000,0.759705,0.741280,0.759705,4.224000e+05,0.0,0.0,MMM,NaN
1962-01-04 00:00:00-05:00,0.000000,0.772462,0.759705,0.759705,2.128000e+05,0.0,0.0,MMM,NaN
1962-01-05 00:00:00-05:00,0.000000,0.756871,0.737027,0.739862,3.152000e+05,0.0,0.0,MMM,NaN
1962-01-08 00:00:00-05:00,0.000000,0.741280,0.731358,0.735610,3.344000e+05,0.0,0.0,MMM,NaN
...,...,...,...,...,...,...,...,...,...
2022-10-31 00:00:00-04:00,3881.850098,3893.729980,3863.179932,3871.979980,4.820620e+09,0.0,0.0,^GSPC,NaN
2022-11-01 00:00:00-04:00,3901.790039,3911.790039,3843.800049,3856.100098,4.481210e+09,0.0,0.0,^GSPC,NaN
2022-11-02 00:00:00-04:00,3852.899902,3894.439941,3758.679932,3759.689941,4.899000e+09,0.0,0.0,^GSPC,NaN


In [7]:
market = data[data['Ticker'] == '^GSPC']

In [8]:
data = data[data['Ticker'] != '^GSPC']
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Adj Close
Date,,,,,,,,,
1962-01-02 00:00:00-05:00,0.000000,0.771044,0.748367,0.754036,212800.0,0.000,0.0,MMM,NaN
1962-01-03 00:00:00-05:00,0.000000,0.759705,0.741280,0.759705,422400.0,0.000,0.0,MMM,NaN
1962-01-04 00:00:00-05:00,0.000000,0.772462,0.759705,0.759705,212800.0,0.000,0.0,MMM,NaN
1962-01-05 00:00:00-05:00,0.000000,0.756871,0.737027,0.739862,315200.0,0.000,0.0,MMM,NaN
1962-01-08 00:00:00-05:00,0.000000,0.741280,0.731358,0.735610,334400.0,0.000,0.0,MMM,NaN
...,...,...,...,...,...,...,...,...,...
2022-10-31 00:00:00-04:00,152.110001,153.339996,149.839996,150.779999,1899000.0,0.325,0.0,ZTS,NaN
2022-11-01 00:00:00-04:00,151.399994,154.910004,150.380005,152.720001,2286200.0,0.000,0.0,ZTS,NaN
2022-11-02 00:00:00-04:00,152.869995,153.210007,147.190002,147.360001,2638300.0,0.000,0.0,ZTS,NaN


In [9]:
market['market return'] = market['Close'].pct_change()
market

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Adj Close,market return
Date,,,,,,,,,,
1950-01-03 00:00:00-05:00,16.660000,16.660000,16.660000,16.660000,1.260000e+06,0.0,0.0,^GSPC,NaN,NaN
1950-01-04 00:00:00-05:00,16.850000,16.850000,16.850000,16.850000,1.890000e+06,0.0,0.0,^GSPC,NaN,0.011405
1950-01-05 00:00:00-05:00,16.930000,16.930000,16.930000,16.930000,2.550000e+06,0.0,0.0,^GSPC,NaN,0.004748
1950-01-06 00:00:00-05:00,16.980000,16.980000,16.980000,16.980000,2.010000e+06,0.0,0.0,^GSPC,NaN,0.002953
1950-01-09 00:00:00-05:00,17.080000,17.080000,17.080000,17.080000,2.520000e+06,0.0,0.0,^GSPC,NaN,0.005889
...,...,...,...,...,...,...,...,...,...,...
2022-10-31 00:00:00-04:00,3881.850098,3893.729980,3863.179932,3871.979980,4.820620e+09,0.0,0.0,^GSPC,NaN,-0.007454
2022-11-01 00:00:00-04:00,3901.790039,3911.790039,3843.800049,3856.100098,4.481210e+09,0.0,0.0,^GSPC,NaN,-0.004101
2022-11-02 00:00:00-04:00,3852.899902,3894.439941,3758.679932,3759.689941,4.899000e+09,0.0,0.0,^GSPC,NaN,-0.025002


In [10]:
data = pd.merge(data, market[['market return']], how='left', left_index=True, right_index=True)
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Adj Close,market return
Date,,,,,,,,,,
1962-01-02 00:00:00-05:00,0.000000,0.771044,0.748367,0.754036,212800.0,0.0,0.0,MMM,NaN,-0.008246
1962-01-02 00:00:00-05:00,0.000000,0.003186,0.003142,0.003142,345600.0,0.0,0.0,MO,NaN,-0.008246
1962-01-02 00:00:00-05:00,0.000000,1.043712,1.019569,1.019569,5800.0,0.0,0.0,AEP,NaN,-0.008246
1962-01-02 00:00:00-05:00,0.194272,0.194272,0.190931,0.190931,352350.0,0.0,0.0,BA,NaN,-0.008246
1962-01-02 00:00:00-05:00,0.488671,0.493430,0.483911,0.488671,163200.0,0.0,0.0,CAT,NaN,-0.008246
...,...,...,...,...,...,...,...,...,...,...
2022-11-04 00:00:00-04:00,121.150002,122.419998,120.250000,121.260002,1670769.0,0.0,0.0,YUM,NaN,0.004573
2022-11-04 00:00:00-04:00,231.830002,232.630005,224.865005,228.014999,272827.0,0.0,0.0,ZBRA,NaN,0.004573
2022-11-04 00:00:00-04:00,106.610001,107.500000,103.750000,104.470001,964495.0,0.0,0.0,ZBH,NaN,0.004573


In [11]:
data = data.sort_values(['Ticker', 'Date']).reset_index()
data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Adj Close,market return
0,1999-11-18 00:00:00-05:00,27.847825,30.602008,24.481605,26.929766,62546380.0,0.000,0.0,A,NaN,0.010087
1,1999-11-19 00:00:00-05:00,26.279474,26.317727,24.366850,24.711123,15234146.0,0.000,0.0,A,NaN,-0.002063
2,1999-11-22 00:00:00-05:00,25.284907,26.929766,24.519857,26.929766,6577870.0,0.000,0.0,A,NaN,-0.000745
3,1999-11-23 00:00:00-05:00,26.011702,26.700248,24.481602,24.481602,5975611.0,0.000,0.0,A,NaN,-0.011471
4,1999-11-24 00:00:00-05:00,24.558100,25.667423,24.481594,25.131887,4843231.0,0.000,0.0,A,NaN,0.008856
...,...,...,...,...,...,...,...,...,...,...,...
4122979,2022-10-31 00:00:00-04:00,152.110001,153.339996,149.839996,150.779999,1899000.0,0.325,0.0,ZTS,NaN,-0.007454
4122980,2022-11-01 00:00:00-04:00,151.399994,154.910004,150.380005,152.720001,2286200.0,0.000,0.0,ZTS,NaN,-0.004101
4122981,2022-11-02 00:00:00-04:00,152.869995,153.210007,147.190002,147.360001,2638300.0,0.000,0.0,ZTS,NaN,-0.025002
4122982,2022-11-03 00:00:00-04:00,134.149994,134.149994,124.150002,131.139999,8065500.0,0.000,0.0,ZTS,NaN,-0.010586


In [12]:
data.to_csv("price.csv")